In [1]:
import sqlite3
import string
from matplotlib import pyplot as plt

database_path = '/home/myrto/pink_dir/github/DataFiles/december_17_db.db'

conn = sqlite3.connect(database_path)
cursor = conn.cursor()

In [2]:
cursor.execute("""SELECT * FROM comment_pairs""")
ps = cursor.fetchall()
sum_of_comment_pairs = len(ps)

cursor.execute("""SELECT subreddit, COUNT(subreddit) FROM comment_pairs GROUP BY subreddit ORDER BY COUNT(subreddit) DESC""")
ps = cursor.fetchall()
print('Subreddits are '+ str(len(ps)) +' with '+ str(sum_of_comment_pairs)+ ' comment pairs.')

Subreddits are 17046 with 3701464 comment pairs.


# Threshold

Τα συνολικά subreddits είναι 17.046 με 3.701.464 ζεύγη σχολίων. Είναι λογικό να σκεφτούμε, πως για να βγάλουμε κάποιο συμπέρασμα για τη θεματολογία του subreddit πρέπει να έχουμε αρκετά σχόλια που να ανήκουν σε αυτό.

In [3]:
cursor.execute("""SELECT subreddit, COUNT(subreddit) FROM comment_pairs GROUP BY subreddit HAVING COUNT(subreddit)<10""")
ps = cursor.fetchall()
print('Subreddits with less than 10 comment pairs: ' + str(len(ps)))

Subreddits with less than 10 comment pairs: 9690


Παρατηρούμε, ωστόσο, ότι παραπάνω από τα μισά subreddits δεν έχουν ούτε 10 ζεύγη σχολίων, πράγμα που μας οδηγεί στο να βάλουμε ένα κατώφλι στα subreddits ώστε να απορρίπτουμε εκείνα με μικρότερο πλήθος σχολίων από αυτό. Επιλέξαμε τα 1.000 ζεύγη σχολίων.

In [4]:
cursor.execute("""SELECT subreddit, COUNT(subreddit) FROM comment_pairs GROUP BY subreddit HAVING COUNT(subreddit)>1000 ORDER BY COUNT(subreddit) DESC""")
big_enough_subreddits = cursor.fetchall()

subreddits, count_comments = map(list, zip(*big_enough_subreddits))
print('There are ' + str(len(big_enough_subreddits)) +' subreddits to be used with total number of comments: ' +str(sum(count_comments)))

There are 638 subreddits to be used with total number of comments: 2810744


In [5]:
# import pandas as pd
# sb = pd.DataFrame(subreddits[0:15], columns = ['Subreddit'])
# cc = pd.DataFrame(count_comments[0:15], columns = ['Comment pairs'])
# pd.concat([sb,cc], axis = 1)

# Data Tokenization

In [6]:
cursor.execute("""SELECT p.parent_body, p.reply_body, p.subreddit FROM comment_pairs p JOIN
                ( SELECT subreddit FROM comment_pairs GROUP BY subreddit HAVING COUNT(subreddit) > 1000 ) b 
                ON p.subreddit = b.subreddit""")

ps = cursor.fetchall()

In [11]:
file = open('data.train', 'w', buffering=10000)
labels = open('labels.train', 'w', buffering=10000)

for parent_body, reply_body, subreddit in ps:
    file.write(parent_body + reply_body + '\n')
    labels.write(subreddit + '\n')       

file.close()
labels.close()

Στην συνέχεια, το data.train μπαίνει ως αρχείο εκπαίδευσης στο new_data του nmt-chatbot, γίνεται tokenizarion και το παίρνουμε ωε train.bpe.from από τον φάκελο data.